### MTA Turnstile data: MVP
The goal of the project is to predict the stations with the highest traffic in the upcoming Summer based on the data from previous Summers. The data of June, July, and August in 2019 was loaded into Python to explored. (the data of 2020 was purposely ignored because of the pandamic impact on the patterns in data)

### Data Cleaning

In [2]:
import pandas as pd

In [11]:
# load the data of Summer 2019 from URL

def get_data_parse_dt(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url, parse_dates=[['DATE','TIME']], keep_date_col=True))
    return pd.concat(dfs)
        
week_nums = [190601, 190608, 190615, 190622, 190629, 190706, 190713,
             190720, 190727, 190803, 190810, 190817, 190824, 190831]
turnstiles_df = get_data_parse_dt(week_nums)

In [12]:
turnstiles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2893420 entries, 0 to 205262
Data columns (total 12 columns):
 #   Column                                                                Dtype         
---  ------                                                                -----         
 0   DATE_TIME                                                             datetime64[ns]
 1   C/A                                                                   object        
 2   UNIT                                                                  object        
 3   SCP                                                                   object        
 4   STATION                                                               object        
 5   LINENAME                                                              object        
 6   DIVISION                                                              object        
 7   DATE                                                                  obj

In [14]:
# remove whitespaces from columns names
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]
turnstiles_df.columns

Index(['DATE_TIME', 'C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION',
       'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [40]:
# sanity check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique for ENTRIES
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values(["ENTRIES", "STATION"], ascending=False)).head(55)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
1796773,R101,R001,02-00-00,SOUTH FERRY,2019-08-22 05:00:00,2
2388681,R290,R161,00-00-00,KINGSBRIDGE RD,2019-06-07 05:00:00,2
2790743,R622,R123,00-00-06,FRANKLIN AV,2019-08-22 04:59:02,2
2791345,R622,R123,00-00-07,FRANKLIN AV,2019-08-22 04:59:02,2
385776,E001,R368,00-00-01,9 AV,2019-07-30 05:00:00,2
2320233,R249,R179,01-05-00,86 ST,2019-06-27 00:00:00,2
2320237,R249,R179,01-05-00,86 ST,2019-06-27 04:00:00,2
2320241,R249,R179,01-05-00,86 ST,2019-06-27 08:00:00,2
2320244,R249,R179,01-05-00,86 ST,2019-06-27 12:00:00,2
684966,N045,R187,01-00-00,81 ST-MUSEUM,2019-06-01 17:00:00,2


In [42]:
import datetime

# we have some duplicated entries and exists for same time. Let's check some entries to understand the error
mask = ((turnstiles_df["C/A"] == "R101") & 
(turnstiles_df["UNIT"] == "R001") & 
(turnstiles_df["SCP"] == "02-00-00") & 
(turnstiles_df["STATION"] == "SOUTH FERRY") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2019, 8, 22).date()))

turnstiles_df[mask].head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
127833,2019-08-22 01:00:00,R101,R001,02-00-00,SOUTH FERRY,1RW,IRT,08/22/2019,01:00:00,REGULAR,3559356,5452274
127834,2019-08-22 05:00:00,R101,R001,02-00-00,SOUTH FERRY,1RW,IRT,08/22/2019,05:00:00,REGULAR,3559385,5452303
127835,2019-08-22 05:00:00,R101,R001,02-00-00,SOUTH FERRY,1RW,IRT,08/22/2019,05:00:00,RECOVR AUD,3559384,5452303
127836,2019-08-22 09:00:00,R101,R001,02-00-00,SOUTH FERRY,1RW,IRT,08/22/2019,09:00:00,REGULAR,3559868,5452791
127837,2019-08-22 13:00:00,R101,R001,02-00-00,SOUTH FERRY,1RW,IRT,08/22/2019,13:00:00,REGULAR,3560128,5453294


In [43]:
mask = ((turnstiles_df["C/A"] == "E001") & 
(turnstiles_df["UNIT"] == "R368") & 
(turnstiles_df["SCP"] == "00-00-01") & 
(turnstiles_df["STATION"] == "9 AV") &
(turnstiles_df["DATE_TIME"].dt.date == datetime.datetime(2019, 7, 30).date()))

turnstiles_df[mask].head()

,DATE_TIME,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
27808,2019-07-30 01:00:00,E001,R368,00-00-01,9 AV,D,BMT,07/30/2019,01:00:00,REGULAR,3499282,2678138
27809,2019-07-30 05:00:00,E001,R368,00-00-01,9 AV,D,BMT,07/30/2019,05:00:00,REGULAR,3499299,2678143
27810,2019-07-30 05:00:00,E001,R368,00-00-01,9 AV,D,BMT,07/30/2019,05:00:00,RECOVR AUD,3499298,2678143
27811,2019-07-30 09:00:00,E001,R368,00-00-01,9 AV,D,BMT,07/30/2019,09:00:00,REGULAR,3499692,2678239
27812,2019-07-30 13:00:00,E001,R368,00-00-01,9 AV,D,BMT,07/30/2019,13:00:00,REGULAR,3499944,2678340


In [46]:
# it seems that the duplicated rows are just missed audits. Let's drop them
turnstiles_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True, ascending=False)
turnstiles_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [47]:
# sanity check to verify that all duplicated entries are removed
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values(["ENTRIES", "STATION"], ascending=False)).head(55)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
2542733,R419,R326,00-00-00,ZEREGA AV,2019-05-25 01:00:00,1
2542734,R419,R326,00-00-00,ZEREGA AV,2019-05-25 05:00:00,1
2542735,R419,R326,00-00-00,ZEREGA AV,2019-05-25 09:00:00,1
2542736,R419,R326,00-00-00,ZEREGA AV,2019-05-25 13:00:00,1
2542737,R419,R326,00-00-00,ZEREGA AV,2019-05-25 17:00:00,1
2542738,R419,R326,00-00-00,ZEREGA AV,2019-05-25 21:00:00,1
2542739,R419,R326,00-00-00,ZEREGA AV,2019-05-26 01:00:00,1
2542740,R419,R326,00-00-00,ZEREGA AV,2019-05-26 05:00:00,1
2542741,R419,R326,00-00-00,ZEREGA AV,2019-05-26 09:00:00,1
2542742,R419,R326,00-00-00,ZEREGA AV,2019-05-26 13:00:00,1



*   In the next steps, I will try to calculate the number of traffic on each turnstile in order to find out the stations with the highest traffic. Some graphs might be plotted to visualize the results.

*   Also the datasets will be uploaded into a PostgreSQL database by creating a PostgreSQL server in GCP SQL. Then it will be imported into Python using SQLAlchemy. (already started working on this task)

* After that, a regression model will be trained on the datasets to generate predictions.  